In [29]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen


In [32]:
url = "./chess_results.html"

with open(url) as fp:
    soup = BeautifulSoup(fp, 'html.parser')

In [56]:
# using soup to select by css selectors
# https://stackoverflow.com/questions/24801548/how-to-use-css-selectors-to-retrieve-specific-links-lying-in-some-class-using-be
table = soup.select('table.CRs2 tr')

In [94]:
tournaments = {
    'offline': [],
    'online': []
}

In [95]:
loc_idx = 8
url_idx = 0

# ignore first row
for row in table[1:]:
    
    # Get values in row
    vals = row.select('td')
    
    # Get url
    # Kinda guessed this
    url = vals[url_idx].a['href']
    
    # Get location
    location = vals[loc_idx].text
    
    # See if online or offline or neither
    medium = None
    if '.com' in location.lower() or 'online' in location.lower():
        tournaments['online'].append(url)
    elif location != '':
        tournaments['offline'].append(url)

    print('{0} : {1}'.format(url, location))

https://chess-results.com/tnr686540.aspx?lan=1 : Mellieha, Malta
https://chess-results.com/tnr669871.aspx?lan=1 : Mellieha, Malta
https://chess-results.com/tnr686538.aspx?lan=1 : Mellieha, Malta
https://chess-results.com/tnr686539.aspx?lan=1 : Mellieha, Malta
https://chess-results.com/tnr670407.aspx?lan=1 : Cala Gonone - Dorgali (NU)
https://chess-results.com/tnr670410.aspx?lan=1 : Cala Gonone - Dorgali (NU)
https://chess-results.com/tnr683767.aspx?lan=1 : Spain / Torrevieja
https://chess-results.com/tnr665934.aspx?lan=1 : Spain / Torrevieja
https://chess-results.com/tnr680077.aspx?lan=1 : Nakhchivan
https://chess-results.com/tnr680641.aspx?lan=1 : Crete
https://chess-results.com/tnr680633.aspx?lan=1 : Crete
https://chess-results.com/tnr680631.aspx?lan=1 : Crete
https://chess-results.com/tnr680636.aspx?lan=1 : Crete
https://chess-results.com/tnr680637.aspx?lan=1 : Crete
https://chess-results.com/tnr680639.aspx?lan=1 : Crete
https://chess-results.com/tnr680944.aspx?lan=1 : Nakhchivan
ht

In [96]:
len(tournaments['online'])

98

In [97]:
len(tournaments['offline'])

215

In [27]:
test_url = f_links[0]

In [30]:
# https://pythonprogramminglanguage.com/get-links-from-webpage/
req = Request(test_url)
html_page = urlopen(req)

In [31]:
soup = BeautifulSoup(test_url, 'html.parser')

In [ ]:
# Get attribute data ...